In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install basedosdados==1.6.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import basedosdados as bd

In [ ]:
#cria df com sigla_uf e nome dos municípios para substituição do nome da capital por id_municipio
query = '''
SELECT sigla_uf, nome, id_municipio FROM basedosdados.br_bd_diretorios_brasil.municipio 
WHERE capital_uf = 1
'''
municipio = bd.read_sql(query, billing_project_id='my-project-51209')

Downloading: 100%|██████████| 27/27 [00:00<00:00, 94.42rows/s]


#2016

In [ ]:
#coleta dados de 2016
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T18)
df1 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T18', skiprows=4, dtype=str)
df1 = df1[df1['UF'].notna()] #exclui toda linha que possui NA no campo UF
ordem = ['ano', 'sigla_uf', 'id_municipio', 'grupo', 'quantidade_homicidio_doloso']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 'Nº de Vítimas':'quantidade_homicidio_doloso'}
df1.rename(columns=rename, inplace=True)
df1['capital'] = df1['capital'].str.replace(r"\(.*\)","") #o nome do município contém (4) e (5) para indicar notas de rodapé. Exclui-se para dar match no nome substituindo pelo id_municipio
df1['capital'] = df1['capital'].str.strip() #retira espaços em branco (especialmente por causa dessas notas de rodapé)
df1 = pd.merge(df1, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) #match com a base do diretório substituindo o nome por id_municipio
df1['ano'] = '2016' #inclui a informação de ano, já que no dado original está na coluna. as informações de 2017 aqui são dispensáveis, pois no anuário 2017-2018 apresentam-se os dados revisados
df1['grupo'] = df1['grupo'].ffill()
df1 = df1[ordem] #ordena as variáveis segundo lista criada anteriormente

#segunda tabela: latrocinio por número de vítimas (T19)
df2 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T19', skiprows=4, dtype=str)
df2 = df2[df2['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_latrocinio']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 'Nº de Vítimas':'quantidade_latrocinio'}
df2.rename(columns=rename, inplace=True)
df2['quantidade_latrocinio'] = df2['quantidade_latrocinio'].replace('-', 0) #o hífen significa que o fenômeno não existiu no dado ano na capital do estado
df2['quantidade_latrocinio'] = df2['quantidade_latrocinio'].replace('...', np.nan) #os três pontos significam que a informação não está disponível
df2['quantidade_latrocinio'] = df2['quantidade_latrocinio'].astype(int)
df2['ano'] = '2016'
df2 = df2[ordem]

df3 = pd.merge(df1, df2, how='left', on=['ano', 'sigla_uf']) #a necessidade de tratar o nome do município nas seguintes é desnecessária pois basta um merge por sigla_uf para trazer a info da tabela ao conjunto completo (df)

#terceira tabela: lesão corporal seguida de morte (T20)
df4 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T20', skiprows=4, dtype=str)
df4 = df4[df4['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 'Nº de Vítimas':'quantidade_lesao_corporal_morte'}
df4.rename(columns=rename, inplace=True)
df4['quantidade_lesao_corporal_morte'] = df4['quantidade_lesao_corporal_morte'].replace('-', 0) 
df4['quantidade_lesao_corporal_morte'] = df4['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df4['quantidade_lesao_corporal_morte'] = df4['quantidade_lesao_corporal_morte'].astype(int) 
df4['ano'] = '2016'
df4 = df4[ordem]

df5 = pd.merge(df3, df4, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Policiais Civis e Militates mortos em confronto em serviço ou em confronto ou por lesão não natural fora de serviço(T21)
df6 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T21', skiprows=4, dtype=str)
df6 = df6[df6['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_morte_policiais_civis_confronto_em_servico',
         'quantidade_morte_policiais_militares_confronto_em_servico', 
         'quantidade_morte_policiais_civis_fora_de_servico', 
         'quantidade_morte_policiais_militares_fora_de_servico',
         'quantidade_mortes_policiais_confronto']
rename = {'Grupos segundo qualidade dos dados (2)':'grupo', 'UF':'sigla_uf', 
          'Capitais':'capital', 
          'Policiais Civis mortos em confronto em serviço ':'quantidade_morte_policiais_civis_confronto_em_servico', 
          'Policiais Militares mortos em confronto em serviço ':'quantidade_morte_policiais_militares_confronto_em_servico', 
          'Policiais Civis mortos em confronto ou por lesão não natural fora de serviço ':'quantidade_morte_policiais_civis_fora_de_servico', 
          'Policiais Militares mortos em confronto ou por lesão não natural fora de serviço ':'quantidade_morte_policiais_militares_fora_de_servico',
          'Total': 'quantidade_mortes_policiais_confronto'}
df6.rename(columns=rename, inplace=True)
df6['quantidade_morte_policiais_civis_confronto_em_servico'] = df6['quantidade_morte_policiais_civis_confronto_em_servico'].replace('-', 0) 
df6['quantidade_morte_policiais_militares_confronto_em_servico'] = df6['quantidade_morte_policiais_militares_confronto_em_servico'].replace('-', 0) 
df6['quantidade_morte_policiais_civis_fora_de_servico'] = df6['quantidade_morte_policiais_civis_fora_de_servico'].replace('-', 0) 
df6['quantidade_morte_policiais_militares_fora_de_servico'] = df6['quantidade_morte_policiais_militares_fora_de_servico'].replace('-', 0) 
df6['quantidade_mortes_policiais_confronto'] = df6['quantidade_mortes_policiais_confronto'].replace('-', 0) 

df6['quantidade_morte_policiais_civis_confronto_em_servico'] = df6['quantidade_morte_policiais_civis_confronto_em_servico'].replace('...', np.nan) 
df6['quantidade_morte_policiais_militares_confronto_em_servico'] = df6['quantidade_morte_policiais_militares_confronto_em_servico'].replace('...', np.nan) 
df6['quantidade_morte_policiais_civis_fora_de_servico'] = df6['quantidade_morte_policiais_civis_fora_de_servico'].replace('...', np.nan) 
df6['quantidade_morte_policiais_militares_fora_de_servico'] = df6['quantidade_morte_policiais_militares_fora_de_servico'].replace('...', np.nan) 
df6['quantidade_mortes_policiais_confronto'] = df6['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 

df6['quantidade_morte_policiais_civis_confronto_em_servico'] = df6['quantidade_morte_policiais_civis_confronto_em_servico'].astype('float').astype('Int64')
df6['quantidade_morte_policiais_militares_confronto_em_servico'] = df6['quantidade_morte_policiais_militares_confronto_em_servico'].astype('float').astype('Int64')
df6['quantidade_morte_policiais_civis_fora_de_servico'] = df6['quantidade_morte_policiais_civis_fora_de_servico'].astype('float').astype('Int64')
df6['quantidade_morte_policiais_militares_fora_de_servico'] = df6['quantidade_morte_policiais_militares_fora_de_servico'].astype('float').astype('Int64')
df6['quantidade_mortes_policiais_confronto'] = df6['quantidade_mortes_policiais_confronto'].astype('float').astype('Int64')

df6['ano'] = '2016'
df6 = df6[ordem]

df7 = pd.merge(df5, df6, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Mortes decorrentes de intervenções policiais, segundo corporação e situação (em serviço e fora de serviço) (T22)
df8 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T22', skiprows=4, dtype=str)
df8 = df8[df8['UF'].notna()]
ordem = ['ano', 'sigla_uf', 
          'quantidade_mortes_intervencao_policial_civil_em_servico',
          'quantidade_mortes_intervencao_policial_militar_em_servico',
          'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
          'quantidade_mortes_intervencao_policial_militar_fora_de_servico']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 
          'Mortes decorrentes de intervenções de Policiais Civis em serviço':'quantidade_mortes_intervencao_policial_civil_em_servico',
          'Mortes decorrentes de intervenções de Policiais Militares em serviço':'quantidade_mortes_intervencao_policial_militar_em_servico',
          'Mortes decorrentes de intervenções de Policiais Civis fora de serviço':'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
          'Mortes decorrentes de intervenções de Policiais Militares fora de serviço':'quantidade_mortes_intervencao_policial_militar_fora_de_servico'}
df8.rename(columns=rename, inplace=True)
df8['quantidade_mortes_intervencao_policial_civil_em_servico'] = df8['quantidade_mortes_intervencao_policial_civil_em_servico'].replace('-', 0)
df8['quantidade_mortes_intervencao_policial_militar_em_servico'] = df8['quantidade_mortes_intervencao_policial_militar_em_servico'].replace('-', 0)
df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].replace('-', 0)
df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].replace('-', 0)

df8['quantidade_mortes_intervencao_policial_civil_em_servico'] = df8['quantidade_mortes_intervencao_policial_civil_em_servico'].replace('...', np.nan)
df8['quantidade_mortes_intervencao_policial_militar_em_servico'] = df8['quantidade_mortes_intervencao_policial_militar_em_servico'].replace('...', np.nan)
df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].replace('...', np.nan)
df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].replace('...', np.nan)

df8['quantidade_mortes_intervencao_policial_civil_em_servico'] = df8['quantidade_mortes_intervencao_policial_civil_em_servico'].astype('float').astype('Int64')
df8['quantidade_mortes_intervencao_policial_militar_em_servico'] = df8['quantidade_mortes_intervencao_policial_militar_em_servico'].astype('float').astype('Int64')
df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].astype('float').astype('Int64')
df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df8['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].astype('float').astype('Int64')
df8['ano'] = '2016'
df8 = df8[ordem]

df9 = pd.merge(df7, df8, how='left', on=['ano', 'sigla_uf'])

#sexta tabela:  Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T23)
df10 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T23', skiprows=4, dtype=str)
df10 = df10[df10['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 'quantidade_mortes_intervencao_policial', 'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 
          'Mortes Violentas Intencionais - MVI ':'quantidade_mortes_violentas_intencionais',
          'Morte Decorrente de Intervenções Policiais (MDIP) em serviço e fora de serviço':'quantidade_mortes_intervencao_policial',
          'Proporção de MDIP em relação às MVI':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'}
df10.rename(columns=rename, inplace=True)
df10['quantidade_mortes_violentas_intencionais'] = df10['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df10['quantidade_mortes_intervencao_policial'] = df10['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df10['quantidade_mortes_violentas_intencionais'] = df10['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df10['quantidade_mortes_intervencao_policial'] = df10['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df10['quantidade_mortes_violentas_intencionais'] = df10['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df10['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df10['quantidade_mortes_intervencao_policial'] = df10['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df10['ano'] = '2016'
df10 = df10[ordem]

df11 = pd.merge(df9, df10, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Roubo e furto de veículos (T24)
df12 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T24', skiprows=4, dtype=str)
df12 = df12[df12['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_roubo_veiculos', 
          'quantidade_furto_veiculos',
          'quantidade_roubo_furto_veiculos']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais':'capital', 
          'Roubo de veículo':'quantidade_roubo_veiculos',
          'Furto de veículo':'quantidade_furto_veiculos',
          'Roubo e furto de veículo':'quantidade_roubo_furto_veiculos'}
df12.rename(columns=rename, inplace=True)
df12['quantidade_roubo_veiculos'] = df12['quantidade_roubo_veiculos'].replace('-', 0) 
df12['quantidade_furto_veiculos'] = df12['quantidade_furto_veiculos'].replace('-', 0) 
df12['quantidade_roubo_furto_veiculos'] = df12['quantidade_roubo_furto_veiculos'].replace('-', 0) 

df12['quantidade_roubo_veiculos'] = df12['quantidade_roubo_veiculos'].replace('...', np.nan) 
df12['quantidade_furto_veiculos'] = df12['quantidade_furto_veiculos'].replace('...', np.nan) 
df12['quantidade_roubo_furto_veiculos'] = df12['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df12['quantidade_roubo_veiculos'] = df12['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df12['quantidade_furto_veiculos'] = df12['quantidade_furto_veiculos'].astype('float').astype('Int64')
df12['quantidade_roubo_furto_veiculos'] = df12['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')
df12['ano'] = '2016'
df12 = df12[ordem]

df13 = pd.merge(df11, df12, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T25)
df14 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2016-2017.xlsx', sheet_name='T25', skiprows=4, dtype=str)
df14 = df14[df14['UF'].notna()]
ordem = ['ano', 
          'sigla_uf',
          'quantidade_estupro' 
          ]
rename = {'UF':'sigla_uf', 
          'Capitais':'capital', 
          'Estupro':'quantidade_estupro'
          }
df14.rename(columns=rename, inplace=True)
df14['quantidade_estupro'] = df14['quantidade_estupro'].replace('-', 0) 
df14['quantidade_estupro'] = df14['quantidade_estupro'].replace('...', np.nan) 
df14['quantidade_estupro'] = df14['quantidade_estupro'].astype('float').astype('Int64') 

df14['ano'] = '2016'
df14 = df14[ordem]
df15 = pd.merge(df13, df14, how='left', on=['ano', 'sigla_uf'])
df15

,ano,sigla_uf,id_municipio,grupo,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_morte_policiais_civis_confronto_em_servico,quantidade_morte_policiais_militares_confronto_em_servico,quantidade_morte_policiais_civis_fora_de_servico,...,quantidade_mortes_intervencao_policial_militar_em_servico,quantidade_mortes_intervencao_policial_civil_fora_de_servico,quantidade_mortes_intervencao_policial_militar_fora_de_servico,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_estupro
0,2016,AL,2704302,Grupo 1,449,21,2,0,2,1,...,57,0,2,531,59,11,1233,305,1538,155
1,2016,CE,2304400,Grupo 1,965,27,15,0,1,2,...,23,0,7,1046,39,3,6415,2820,9235,458
2,2016,ES,3205309,Grupo 1,51,0,3,0,0,0,...,6,0,2,63,9,14,227,338,565,25
3,2016,GO,5208707,Grupo 1,452,33,14,0,0,0,...,76,0,24,599,100,16,7298,3733,11031,161
4,2016,MA,2111300,Grupo 1,498,23,12,0,0,0,...,27,0,0,560,27,4,1678,487,2165,224
5,2016,MT,5103403,Grupo 1,196,18,4,0,1,0,...,<NA>,0,<NA>,218,NaN,<NA>,1430,1158,2588,353
6,2016,PA,1501402,Grupo 1,878,41,20,0,1,2,...,<NA>,<NA>,<NA>,978,39,3,1904,1012,2916,481
7,2016,PB,2507507,Grupo 1,341,8,1,0,2,1,...,4,0,0,350,5,1,<NA>,<NA>,813,92
8,2016,PE,2611606,Grupo 1,624,5,11,0,0,1,...,<NA>,<NA>,<NA>,640,15,2,3461,1520,4981,446
9,2016,PI,2211001,Grupo 1,341,22,4,0,0,0,...,2,1,4,367,7,1,2483,1104,3587,226


In [ ]:
df15.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2016.csv', index=False)

#2017

In [ ]:
#coleta dados de 2017
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T20)
df16 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T20', skiprows=4, dtype=str)
df16 = df16[df16['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'id_municipio', 'grupo', 'quantidade_homicidio_doloso']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Homicídios dolosos':'quantidade_homicidio_doloso'}
df16.rename(columns=rename, inplace=True)
df16['capital'] = df16['capital'].str.replace(r"\(.*\)","") 
df16['capital'] = df16['capital'].str.strip() 
df16 = pd.merge(df16, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) 
df16['ano'] = '2017'
df16['grupo'] = df16['grupo'].ffill()
df16 = df16[ordem] 

#segunda tabela: latrocinio por número de vítimas (T21)
df17 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T21', skiprows=4, dtype=str)
df17 = df17[df17['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_latrocinio']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Latrocínio':'quantidade_latrocinio'}
df17.rename(columns=rename, inplace=True)
df17['quantidade_latrocinio'] = df17['quantidade_latrocinio'].astype(int)
df17['ano'] = '2017'
df17 = df17[ordem]

df18 = pd.merge(df16, df17, how='left', on=['ano', 'sigla_uf'])

#terceira tabela: lesão corporal seguida de morte (T22)
df19 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T22', skiprows=4, dtype=str)
df19 = df19[df19['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'Grupos segundo qualidade dos dados (1)':'grupo', 'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal seguida de morte':'quantidade_lesao_corporal_morte'}
df19.rename(columns=rename, inplace=True)
df19['quantidade_lesao_corporal_morte'] = df19['quantidade_lesao_corporal_morte'].replace('-', 0) 
df19['quantidade_lesao_corporal_morte'] = df19['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df19['quantidade_lesao_corporal_morte'] = df19['quantidade_lesao_corporal_morte'].astype(int) 
df19['ano'] = '2017'
df19 = df19[ordem]

df20 = pd.merge(df18, df19, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Policiais Civis e Militates mortos em confronto em serviço ou em confronto ou por lesão não natural fora de serviço(T23)
df21 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T23', skiprows=4, dtype=str)
df21 = df21[df21['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_morte_policiais_civis_confronto_em_servico',
         'quantidade_morte_policiais_militares_confronto_em_servico', 
         'quantidade_morte_policiais_civis_fora_de_servico', 
         'quantidade_morte_policiais_militares_fora_de_servico',
         'quantidade_mortes_policiais_confronto']
rename = {'Grupos segundo qualidade dos dados (2)':'grupo', 'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Policiais Civis mortos em confronto em serviço ':'quantidade_morte_policiais_civis_confronto_em_servico', 
          'Policiais Militares mortos em confronto em serviço ':'quantidade_morte_policiais_militares_confronto_em_servico', 
          'Policiais Civis mortos em confronto ou por lesão não natural fora de serviço ':'quantidade_morte_policiais_civis_fora_de_servico', 
          'Policiais Militares mortos em confronto ou por lesão não natural fora de serviço ':'quantidade_morte_policiais_militares_fora_de_servico',
          'Total': 'quantidade_mortes_policiais_confronto'}
df21.rename(columns=rename, inplace=True)
df21['quantidade_morte_policiais_civis_confronto_em_servico'] = df21['quantidade_morte_policiais_civis_confronto_em_servico'].replace('-', 0) 
df21['quantidade_morte_policiais_militares_confronto_em_servico'] = df21['quantidade_morte_policiais_militares_confronto_em_servico'].replace('-', 0) 
df21['quantidade_morte_policiais_civis_fora_de_servico'] = df21['quantidade_morte_policiais_civis_fora_de_servico'].replace('-', 0) 
df21['quantidade_morte_policiais_militares_fora_de_servico'] = df21['quantidade_morte_policiais_militares_fora_de_servico'].replace('-', 0) 
df21['quantidade_mortes_policiais_confronto'] = df21['quantidade_mortes_policiais_confronto'].replace('-', 0) 

df21['quantidade_morte_policiais_civis_confronto_em_servico'] = df21['quantidade_morte_policiais_civis_confronto_em_servico'].replace('...', np.nan) 
df21['quantidade_morte_policiais_militares_confronto_em_servico'] = df21['quantidade_morte_policiais_militares_confronto_em_servico'].replace('...', np.nan) 
df21['quantidade_morte_policiais_civis_fora_de_servico'] = df21['quantidade_morte_policiais_civis_fora_de_servico'].replace('...', np.nan) 
df21['quantidade_morte_policiais_militares_fora_de_servico'] = df21['quantidade_morte_policiais_militares_fora_de_servico'].replace('...', np.nan) 
df21['quantidade_mortes_policiais_confronto'] = df21['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 

df21['quantidade_morte_policiais_civis_confronto_em_servico'] = df21['quantidade_morte_policiais_civis_confronto_em_servico'].astype('float').astype('Int64')
df21['quantidade_morte_policiais_militares_confronto_em_servico'] = df21['quantidade_morte_policiais_militares_confronto_em_servico'].astype('float').astype('Int64')
df21['quantidade_morte_policiais_civis_fora_de_servico'] = df21['quantidade_morte_policiais_civis_fora_de_servico'].astype('float').astype('Int64')
df21['quantidade_morte_policiais_militares_fora_de_servico'] = df21['quantidade_morte_policiais_militares_fora_de_servico'].astype('float').astype('Int64')
df21['quantidade_mortes_policiais_confronto'] = df21['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 

df21['ano'] = '2017'
df21 = df21[ordem]

df22 = pd.merge(df20, df21, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Mortes decorrentes de intervenções policiais, segundo corporação e situação (em serviço e fora de serviço) (T24)
df23 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T24', skiprows=4, dtype=str)
df23 = df23[df23['UF'].notna()]
ordem = ['ano', 'sigla_uf', 
          'quantidade_mortes_intervencao_policial_civil_em_servico',
          'quantidade_mortes_intervencao_policial_militar_em_servico',
          'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
          'quantidade_mortes_intervencao_policial_militar_fora_de_servico']
rename = {'Grupos segundo qualidade dos dados (2)':'grupo', 'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Mortes decorrentes de intervenções de Policiais Civis em serviço':'quantidade_mortes_intervencao_policial_civil_em_servico',
          'Mortes decorrentes de intervenções de Policiais Militares em serviço':'quantidade_mortes_intervencao_policial_militar_em_servico',
          'Mortes decorrentes de intervenções de Policiais Civis fora de serviço':'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
          'Mortes decorrentes de intervenções de Policiais Militares fora de serviço':'quantidade_mortes_intervencao_policial_militar_fora_de_servico'}
df23.rename(columns=rename, inplace=True)
df23['quantidade_mortes_intervencao_policial_civil_em_servico'] = df23['quantidade_mortes_intervencao_policial_civil_em_servico'].replace('-', 0)
df23['quantidade_mortes_intervencao_policial_militar_em_servico'] = df23['quantidade_mortes_intervencao_policial_militar_em_servico'].replace('-', 0)
df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].replace('-', 0)
df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].replace('-', 0)

df23['quantidade_mortes_intervencao_policial_civil_em_servico'] = df23['quantidade_mortes_intervencao_policial_civil_em_servico'].replace('...', np.nan)
df23['quantidade_mortes_intervencao_policial_civil_em_servico'] = df23['quantidade_mortes_intervencao_policial_civil_em_servico'].replace('…', np.nan)
df23['quantidade_mortes_intervencao_policial_militar_em_servico'] = df23['quantidade_mortes_intervencao_policial_militar_em_servico'].replace('...', np.nan)
df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].replace('...', np.nan)
df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].replace('...', np.nan)

df23['quantidade_mortes_intervencao_policial_civil_em_servico'] = df23['quantidade_mortes_intervencao_policial_civil_em_servico'].astype('float').astype('Int64')
df23['quantidade_mortes_intervencao_policial_militar_em_servico'] = df23['quantidade_mortes_intervencao_policial_militar_em_servico'].astype('float').astype('Int64')
df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_civil_fora_de_servico'].astype('float').astype('Int64')
df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'] = df23['quantidade_mortes_intervencao_policial_militar_fora_de_servico'].astype('float').astype('Int64')
df23['ano'] = '2017'
df23 = df23[ordem]

df24 = pd.merge(df22, df23, how='left', on=['ano', 'sigla_uf'])

#sexta tabela: Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T25)
df25 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T25', skiprows=4, dtype=str)
df25 = df25[df25['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 'quantidade_mortes_intervencao_policial', 'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']
rename = {'Grupos segundo qualidade dos dados (2)':'grupo', 'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Mortes Violentas Intencionais - MVI ':'quantidade_mortes_violentas_intencionais',
          'Morte Decorrente de Intervenções Policiais (MDIP) em serviço e fora de serviço':'quantidade_mortes_intervencao_policial',
          'Proporção de MDIP em relação às MVI':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'}
df25.rename(columns=rename, inplace=True)
df25['quantidade_mortes_violentas_intencionais'] = df25['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df25['quantidade_mortes_intervencao_policial'] = df25['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df25['quantidade_mortes_violentas_intencionais'] = df25['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df25['quantidade_mortes_intervencao_policial'] = df25['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df25['quantidade_mortes_violentas_intencionais'] = df25['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df25['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df25['quantidade_mortes_intervencao_policial'] = df25['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 
df25['ano'] = '2017'
df25 = df25[ordem]

df26 = pd.merge(df24, df25, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Roubo e furto de veículos (T26)
df27 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T26', skiprows=4, dtype=str)
df27 = df27[df27['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_roubo_veiculos', 
          'quantidade_furto_veiculos',
          'quantidade_roubo_furto_veiculos']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Roubo de veículo':'quantidade_roubo_veiculos',
          'Furto de veículo':'quantidade_furto_veiculos',
          'Roubo e furto de veículo':'quantidade_roubo_furto_veiculos'}
df27.rename(columns=rename, inplace=True)

df27['quantidade_roubo_veiculos'] = df27['quantidade_roubo_veiculos'].replace('...', np.nan) 
df27['quantidade_furto_veiculos'] = df27['quantidade_furto_veiculos'].replace('...', np.nan) 
df27['quantidade_roubo_furto_veiculos'] = df27['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df27['quantidade_roubo_veiculos'] = df27['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df27['quantidade_furto_veiculos'] = df27['quantidade_furto_veiculos'].astype('float').astype('Int64')
df27['quantidade_roubo_furto_veiculos'] = df27['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')
df27['ano'] = '2017'
df27 = df27[ordem]

df28 = pd.merge(df26, df27, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T27)
df29 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2017-2018.xlsx', sheet_name='T27', skiprows=4, dtype=str)
df29 = df29[df29['UF'].notna()]
ordem = ['ano', 
          'sigla_uf',
          'quantidade_estupro'
          ]
rename = {'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Estupro':'quantidade_estupro'
          }
df29.rename(columns=rename, inplace=True)
df29['quantidade_estupro'] = df29['quantidade_estupro'].replace('-', 0) 
df29['quantidade_estupro'] = df29['quantidade_estupro'].replace('...', np.nan) 
df29['quantidade_estupro'] = df29['quantidade_estupro'].astype('float').astype('Int64') 

df29['ano'] = '2017'
df29 = df29[ordem]
df30 = pd.merge(df28, df29, how='left', on=['ano', 'sigla_uf'])
dfx1 = df30[df30['sigla_uf'] == 'DF'] 
dfx1['id_municipio'] = 5300108
df30[df30['sigla_uf'] == 'DF'] = dfx1
df30

,ano,sigla_uf,id_municipio,grupo,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_morte_policiais_civis_confronto_em_servico,quantidade_morte_policiais_militares_confronto_em_servico,quantidade_morte_policiais_civis_fora_de_servico,...,quantidade_mortes_intervencao_policial_militar_em_servico,quantidade_mortes_intervencao_policial_civil_fora_de_servico,quantidade_mortes_intervencao_policial_militar_fora_de_servico,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_estupro
0,2017,AL,2704302,Grupo 1,590,15,3,0,0,0,...,52,0,3,665,57,8,1057,515,1572,185
1,2017,CE,2304400,Grupo 1,1932,29,18,0,1,1,...,30,3,13,2029,50,2,6531,2734,9265,504
2,2017,ES,3205309,Grupo 1,86,2,0,0,<NA>,0,...,4,0,1,93,5,5,629,816,1445,29
3,2017,GO,5208707,Grupo 1,438,17,5,0,0,0,...,93,0,12,565,105,18,5635,3634,9269,117
4,2017,MA,2111300,Grupo 1,403,17,19,0,0,0,...,34,<NA>,<NA>,473,34,7,1057,539,1596,272
5,2017,MT,5103403,Grupo 1,142,11,0,0,0,0,...,11,0,0,153,11,7,1196,926,2122,320
6,2017,PA,1501402,Grupo 1,875,45,15,0,2,3,...,44,0,6,989,54,5,2181,1356,3537,573
7,2017,PB,2507507,Grupo 1,287,9,2,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,298,0,<NA>,<NA>,<NA>,744,31
8,2017,PE,2611606,Grupo 1,772,16,2,0,2,1,...,<NA>,<NA>,<NA>,790,14,1,3802,1692,5494,432
9,2017,PI,2211001,Grupo 1,291,25,3,0,0,0,...,7,1,6,319,14,4,2205,1143,3348,277


In [ ]:
df30.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2017.csv', index=False)

#2018

In [ ]:
#coleta dados de 2018
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T44)
df31 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T44', skiprows=4, dtype=str)
df31 = df31[df31['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'id_municipio', 'quantidade_homicidio_doloso']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Homicídios dolosos':'quantidade_homicidio_doloso'}
df31.rename(columns=rename, inplace=True)
df31['capital'] = df31['capital'].str.replace(r"\(.*\)","") 
df31['capital'] = df31['capital'].str.strip() 
df31 = pd.merge(df31, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) 
df31['ano'] = '2018'
df31 = df31[ordem] 
x = df31.iloc[27:33,:] #Para tirar as linhas que pegam as informações de Fonte
df31.drop(x.index, inplace=True)

#segunda tabela: latrocinio por número de vítimas (T45)
df32 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T45', skiprows=4, dtype=str)
df32 = df32[df32['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_latrocinio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Latrocínio':'quantidade_latrocinio'}
df32.rename(columns=rename, inplace=True)
df32.reset_index(inplace=True, drop=True) #Para resetar o index
x = df32.iloc[27:32] #Para tirar as linhas que pegam as informações de Fonte
df32.drop(x.index, inplace=True)
df32['quantidade_latrocinio'] = df32['quantidade_latrocinio'].astype(int)
df32['ano'] = '2018'
df32 = df32[ordem]

df33 = pd.merge(df31, df32, how='left', on=['ano', 'sigla_uf'])

#terceira tabela: lesão corporal seguida de morte (T46)
df34 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T46', skiprows=4, dtype=str)
df34 = df34[df34['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal seguida de morte':'quantidade_lesao_corporal_morte'}
df34.rename(columns=rename, inplace=True)
df34['quantidade_lesao_corporal_morte'] = df34['quantidade_lesao_corporal_morte'].replace('-', 0) 
df34['quantidade_lesao_corporal_morte'] = df34['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df34.reset_index(inplace=True, drop=True) #Para resetar o index
x = df34.iloc[27:33] #Para tirar as linhas que pegam as informações de Fonte
df34.drop(x.index, inplace=True)
df34['quantidade_lesao_corporal_morte'] = df34['quantidade_lesao_corporal_morte'].astype(int) 
df34['ano'] = '2018'
df34 = df34[ordem]

df35 = pd.merge(df33, df34, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Mortes decorrentes de intervenções policiais e Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T47)
df36 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T47', skiprows=4, dtype=str)
df36 = df36[df36['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 'quantidade_mortes_intervencao_policial', 'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']

rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Mortes Violentas Intencionais - MVI ':'quantidade_mortes_violentas_intencionais',
          'Proporção de MDIP em relação às MVI':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
          'Morte Decorrente de Intervenções Policiais (MDIP) em serviço e fora de serviço':'quantidade_mortes_intervencao_policial',
          }
df36.rename(columns=rename, inplace=True)

df36['quantidade_mortes_violentas_intencionais'] = df36['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df36['quantidade_mortes_intervencao_policial'] = df36['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df36['quantidade_mortes_violentas_intencionais'] = df36['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df36['quantidade_mortes_intervencao_policial'] = df36['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df36['quantidade_mortes_violentas_intencionais'] = df36['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df36['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df36['quantidade_mortes_intervencao_policial'] = df36['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df36['ano'] = '2018'
df36 = df36[ordem]

df37 = pd.merge(df35, df36, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Policiais Civis e Militares Mortos em Situação de Confronto (T43)
df38 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T43', skiprows=4, dtype=str)
df38 = df38[df38['UF'].notna()]

ordem = ['ano', 'sigla_uf', 'quantidade_mortes_policiais_confronto']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Policiais Civis e Militares Mortos em Situação de Confronto':'quantidade_mortes_policiais_confronto'}
df38.rename(columns=rename, inplace=True)
df38['quantidade_mortes_policiais_confronto'] = df38['quantidade_mortes_policiais_confronto'].replace('-', 0) 
df38['quantidade_mortes_policiais_confronto'] = df38['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 
df38['quantidade_mortes_policiais_confronto'] = df38['quantidade_mortes_policiais_confronto'].astype('float').astype('Int64')

df38['ano'] = '2018'
df38 = df38[ordem]

df39 = pd.merge(df37, df38, how='left', on=['ano', 'sigla_uf'])

#sexta tabela: Feminicídios (T48) 
df40 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T48', skiprows=4, dtype=str)
df40 = df40[df40['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_feminicidio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Feminicídios':'quantidade_feminicidio'}
df40.rename(columns=rename, inplace=True)

df40['quantidade_feminicidio'] = df40['quantidade_feminicidio'].replace('-', 0) 
df40['quantidade_feminicidio'] = df40['quantidade_feminicidio'].replace('...', np.nan) 
df40['quantidade_feminicidio'] = df40['quantidade_feminicidio'].astype('float').astype('Int64') 

df40['ano'] = '2018'
df40 = df40[ordem]

df41 = pd.merge(df39, df40, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Lesão Corporal Dolosa - Violência doméstica (T49)
df42 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T49', skiprows=4, dtype=str)
df42 = df42[df42['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_dolosa_violencia_domestica']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal dolosa - violência doméstica':'quantidade_lesao_corporal_dolosa_violencia_domestica'}
df42.rename(columns=rename, inplace=True)

df42['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df42['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('-', 0) 
df42['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df42['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('...', np.nan) 
df42['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df42['quantidade_lesao_corporal_dolosa_violencia_domestica'].astype('float').astype('Int64') 

df42['ano'] = '2018'
df42 = df42[ordem]

df43 = pd.merge(df41, df42, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T50)
df44 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T50', skiprows=4, dtype=str)
df44 = df44[df44['UF'].notna()]
ordem = ['ano', 
        'sigla_uf',
        'quantidade_estupro']
rename = {'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Estupro':'quantidade_estupro'
          }
df44.rename(columns=rename, inplace=True)
df44['quantidade_estupro'] = df44['quantidade_estupro'].replace('-', 0) 
df44['quantidade_estupro'] = df44['quantidade_estupro'].replace('...', np.nan) 
df44['quantidade_estupro'] = df44['quantidade_estupro'].astype('float').astype('Int64') 

df44['ano'] = '2018'
df44 = df44[ordem]
df45 = pd.merge(df43, df44, how='left', on=['ano', 'sigla_uf'])

#nona tabela: Roubos e Furtos de veículos (T51)
df46 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T51', skiprows=4, dtype=str)
df46 = df46[df46['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_roubo_veiculos', 
          'quantidade_furto_veiculos',
          'quantidade_roubo_furto_veiculos']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Roubo de veículo':'quantidade_roubo_veiculos',
          'Furto de veículo':'quantidade_furto_veiculos',
          'Roubo e furto de veículo':'quantidade_roubo_furto_veiculos'}
df46.rename(columns=rename, inplace=True)

df46['quantidade_roubo_veiculos'] = df46['quantidade_roubo_veiculos'].replace('...', np.nan) 
df46['quantidade_furto_veiculos'] = df46['quantidade_furto_veiculos'].replace('...', np.nan) 
df46['quantidade_roubo_furto_veiculos'] = df46['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df46['quantidade_roubo_veiculos'] = df46['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df46['quantidade_furto_veiculos'] = df46['quantidade_furto_veiculos'].astype('float').astype('Int64')
df46['quantidade_roubo_furto_veiculos'] = df46['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')
df46['ano'] = '2018'
df46 = df46[ordem]

df47 = pd.merge(df45, df46, how='left', on=['ano', 'sigla_uf'])

#décima tabela: Posse e porte ilegal de arma de fogo (T52)
df48 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T52', skiprows=4, dtype=str)
df48 = df48[df48['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_posse_ilegal_arma_de_fogo', 
          'quantidade_porte_ilegal_arma_de_fogo',
          'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Posse ilegal de arma de fogo':'quantidade_posse_ilegal_arma_de_fogo',
          'Porte ilegal de arma de fogo':'quantidade_porte_ilegal_arma_de_fogo',
          'Posse e porte ilegal de arma de fogo':'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'}
df48.rename(columns=rename, inplace=True)

df48['quantidade_posse_ilegal_arma_de_fogo'] = df48['quantidade_posse_ilegal_arma_de_fogo'].replace('...', np.nan) 
df48['quantidade_porte_ilegal_arma_de_fogo'] = df48['quantidade_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 
df48['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df48['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 

df48['quantidade_posse_ilegal_arma_de_fogo'] = df48['quantidade_posse_ilegal_arma_de_fogo'].astype('float').astype('Int64') 
df48['quantidade_porte_ilegal_arma_de_fogo'] = df48['quantidade_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df48['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df48['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df48['ano'] = '2018'
df48 = df48[ordem]

df49 = pd.merge(df47, df48, how='left', on=['ano', 'sigla_uf'])

#décima primeira tabela: Tráfico de entorpecentes e posse e uso de entropocentes (T53)
df50 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2018-2019.xlsx', sheet_name='T53', skiprows=4, dtype=str)
df50 = df50[df50['UF'].notna()]
ordem = ['ano', 'sigla_uf','quantidade_trafico_entorpecente','quantidade_posse_uso_entorpecente']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Tráfico de Entorpecentes':'quantidade_trafico_entorpecente',
          'Posse e Uso de Entorpecentes':'quantidade_posse_uso_entorpecente'}
df50.rename(columns=rename, inplace=True)

df50['quantidade_trafico_entorpecente'] = df50['quantidade_trafico_entorpecente'].replace('...', np.nan) 
df50['quantidade_posse_uso_entorpecente'] = df50['quantidade_posse_uso_entorpecente'].replace('...', np.nan) 

df50['quantidade_trafico_entorpecente'] = df50['quantidade_trafico_entorpecente'].astype('float').astype('Int64') 
df50['quantidade_posse_uso_entorpecente'] = df50['quantidade_posse_uso_entorpecente'].astype('float').astype('Int64')
df50['ano'] = '2018'
df50 = df50[ordem]

df51 = pd.merge(df49, df50, how='left', on=['ano', 'sigla_uf'])
dfx2 = df51[df51['sigla_uf'] == 'DF'] 
dfx2['id_municipio'] = 5300108
df51[df51['sigla_uf'] == 'DF'] = dfx2
df51.head(6)


,ano,sigla_uf,id_municipio,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,quantidade_mortes_policiais_confronto,...,quantidade_lesao_corporal_dolosa_violencia_domestica,quantidade_estupro,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_posse_ilegal_arma_de_fogo,quantidade_porte_ilegal_arma_de_fogo,quantidade_posse_ilegal_porte_ilegal_arma_de_fogo,quantidade_trafico_entorpecente,quantidade_posse_uso_entorpecente
0,2018,AC,1200401,218,9,0,227,NaN,<NA>,<NA>,...,<NA>,48,<NA>,<NA>,<NA>,1432,<NA>,1432,1040,<NA>
1,2018,AL,2704302,439,16,1,456,NaN,<NA>,1,...,738,213,1009,998,2007,144,394,538,880,237
2,2018,AM,1302603,892,52,24,1008,40,3,3,...,2370,988,3086,2257,5343,616,371,987,2438,653
3,2018,AP,1600303,274,12,28,348,34,9,5,...,<NA>,231,338,639,977,<NA>,196,196,260,99
4,2018,BA,2927408,1115,16,13,1243,99,7,8,...,2884,427,5388,1406,6794,30,185,215,1847,1314
5,2018,CE,2304400,1445,13,19,1540,63,4,8,...,2471,478,5033,2345,7378,371,746,1117,3083,1167


In [ ]:
df51.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2018.csv', index=False)

#2019

In [ ]:
#coleta dados de 2019
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T32)
df52 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T32', skiprows=4, dtype=str)
df52 = df52[df52['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'id_municipio', 'quantidade_homicidio_doloso']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Homicídios dolosos':'quantidade_homicidio_doloso'}
df52.rename(columns=rename, inplace=True)
df52['capital'] = df52['capital'].str.replace(r"\(.*\)","") 
df52['capital'] = df52['capital'].str.strip() 
df52 = pd.merge(df52, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) 
df52['ano'] = '2019'
df52 = df52[ordem] 

#segunda tabela: latrocinio por número de vítimas (T33)
df53 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T33', skiprows=4, dtype=str)
df53 = df53[df53['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_latrocinio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Latrocínio':'quantidade_latrocinio'}
df53.rename(columns=rename, inplace=True)
df53.reset_index(inplace=True, drop=True) #Para resetar o index
df53['quantidade_latrocinio'] = df53['quantidade_latrocinio'].replace('-', 0) 
df53['quantidade_latrocinio'] = df53['quantidade_latrocinio'].astype(int)
df53['ano'] = '2019'
df53 = df53[ordem]

df54 = pd.merge(df52, df53, how='left', on=['ano', 'sigla_uf'])

#terceira tabela: lesão corporal seguida de morte (T34)
df55 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T34', skiprows=4, dtype=str)
df55 = df55[df55['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal seguida de morte':'quantidade_lesao_corporal_morte'}
df55.rename(columns=rename, inplace=True)
df55['quantidade_lesao_corporal_morte'] = df55['quantidade_lesao_corporal_morte'].replace('-', 0) 
df55['quantidade_lesao_corporal_morte'] = df55['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df55.reset_index(inplace=True, drop=True) #Para resetar o index
df55['quantidade_lesao_corporal_morte'] = df55['quantidade_lesao_corporal_morte'].astype(int) 
df55['ano'] = '2019'
df55 = df55[ordem]

df56 = pd.merge(df54, df55, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Mortes decorrentes de intervenções policiais e Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T35)
df57 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T35', skiprows=4, dtype=str)
df57 = df57[df57['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 
         'quantidade_mortes_intervencao_policial', 
         'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']

rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Mortes Violentas Intencionais - MVI ':'quantidade_mortes_violentas_intencionais',
          'Proporção de MDIP em relação às MVI':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
          'Morte Decorrente de Intervenções Policiais (MDIP) em serviço e fora de serviço':'quantidade_mortes_intervencao_policial',
          }
df57.rename(columns=rename, inplace=True)

df57['quantidade_mortes_violentas_intencionais'] = df57['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df57['quantidade_mortes_intervencao_policial'] = df57['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df57['quantidade_mortes_violentas_intencionais'] = df57['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df57['quantidade_mortes_intervencao_policial'] = df57['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df57['quantidade_mortes_violentas_intencionais'] = df57['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df57['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df57['quantidade_mortes_intervencao_policial'] = df57['quantidade_mortes_intervencao_policial'].astype('float').astype('Int64')

df57['ano'] = '2019'
df57 = df57[ordem]

df58 = pd.merge(df56, df57, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Policiais Civis e Militares Mortos em Situação de Confronto (T31)
df59 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T31', skiprows=4, dtype=str)
df59 = df59[df59['UF'].notna()]

ordem = ['ano', 'sigla_uf', 'quantidade_mortes_policiais_confronto']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Policiais Civis e Militares Mortos em Situação de Confronto':'quantidade_mortes_policiais_confronto'}
df59.rename(columns=rename, inplace=True)
df59['quantidade_mortes_policiais_confronto'] = df59['quantidade_mortes_policiais_confronto'].replace('-', 0) 
df59['quantidade_mortes_policiais_confronto'] = df59['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 
df59['quantidade_mortes_policiais_confronto'] = df59['quantidade_mortes_policiais_confronto'].astype('float').astype('Int64')

df59['ano'] = '2019'
df59 = df59[ordem]

df60 = pd.merge(df58, df59, how='left', on=['ano', 'sigla_uf'])

#sexta tabela: Feminicídios (T36) 
df61 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T36', skiprows=4, dtype=str)
df61 = df61[df61['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_feminicidio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Feminicídios':'quantidade_feminicidio'}
df61.rename(columns=rename, inplace=True)

df61['quantidade_feminicidio'] = df61['quantidade_feminicidio'].replace('-', 0) 
df61['quantidade_feminicidio'] = df61['quantidade_feminicidio'].replace('...', np.nan) 
df61['quantidade_feminicidio'] = df61['quantidade_feminicidio'].astype('float').astype('Int64') 

df61['ano'] = '2019'
df61 = df61[ordem]

df62 = pd.merge(df60, df61, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Lesão Corporal Dolosa - Violência doméstica (T37)
df63 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T37', skiprows=4, dtype=str)
df63 = df63[df63['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_dolosa_violencia_domestica']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal dolosa - violência doméstica':'quantidade_lesao_corporal_dolosa_violencia_domestica'}
df63.rename(columns=rename, inplace=True)

df63['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df63['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('-', 0) 
df63['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df63['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('...', np.nan) 
df63['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df63['quantidade_lesao_corporal_dolosa_violencia_domestica'].astype('float').astype('Int64')

df63['ano'] = '2019'
df63 = df63[ordem]

df64 = pd.merge(df62, df63, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T38)
df65 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T38', skiprows=4, dtype=str)
df65 = df65[df65['UF'].notna()]
ordem = ['ano', 
        'sigla_uf',
        'quantidade_estupro']
rename = {'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Estupro':'quantidade_estupro'
          }
df65.rename(columns=rename, inplace=True)
df65['quantidade_estupro'] = df65['quantidade_estupro'].replace('-', 0) 
df65['quantidade_estupro'] = df65['quantidade_estupro'].replace('...', np.nan) 
df65['quantidade_estupro'] = df65['quantidade_estupro'].astype('float').astype('Int64') 

df65['ano'] = '2019'
df65 = df65[ordem]
df66 = pd.merge(df64, df65, how='left', on=['ano', 'sigla_uf'])

#nona tabela: Roubos e Furtos de veículos (T39)
df67 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T39', skiprows=4, dtype=str)
df67 = df67[df67['UF'].notna()]
ordem = ['ano', 'sigla_uf',
         'quantidade_roubo_veiculos', 
         'quantidade_furto_veiculos',
         'quantidade_roubo_furto_veiculos']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Roubo de veículo':'quantidade_roubo_veiculos',
          'Furto de veículo':'quantidade_furto_veiculos',
          'Roubo e furto de veículo':'quantidade_roubo_furto_veiculos'}
df67.rename(columns=rename, inplace=True)

df67['quantidade_roubo_veiculos'] = df67['quantidade_roubo_veiculos'].replace('...', np.nan) 
df67['quantidade_furto_veiculos'] = df67['quantidade_furto_veiculos'].replace('...', np.nan) 
df67['quantidade_roubo_furto_veiculos'] = df67['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df67['quantidade_roubo_veiculos'] = df67['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df67['quantidade_furto_veiculos'] = df67['quantidade_furto_veiculos'].astype('float').astype('Int64')
df67['quantidade_roubo_furto_veiculos'] = df67['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')

df67['ano'] = '2019'
df67 = df67[ordem]

df68 = pd.merge(df66, df67, how='left', on=['ano', 'sigla_uf'])

#décima tabela: Posse e porte ilegal de arma de fogo (T40)
df69 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T40', skiprows=4, dtype=str)
df69 = df69[df69['UF'].notna()]
ordem = ['ano', 'sigla_uf',
         'quantidade_posse_ilegal_arma_de_fogo', 
         'quantidade_porte_ilegal_arma_de_fogo',
         'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Posse ilegal de arma de fogo':'quantidade_posse_ilegal_arma_de_fogo',
          'Porte ilegal de arma de fogo':'quantidade_porte_ilegal_arma_de_fogo',
          'Posse e porte ilegal de arma de fogo':'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'}
df69.rename(columns=rename, inplace=True)

df69['quantidade_posse_ilegal_arma_de_fogo'] = df69['quantidade_posse_ilegal_arma_de_fogo'].replace('...', np.nan) 
df69['quantidade_porte_ilegal_arma_de_fogo'] = df69['quantidade_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 
df69['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df69['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 

df69['quantidade_posse_ilegal_arma_de_fogo'] = df69['quantidade_posse_ilegal_arma_de_fogo'].astype('float').astype('Int64') 
df69['quantidade_porte_ilegal_arma_de_fogo'] = df69['quantidade_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df69['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df69['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')

df69['ano'] = '2019'
df69 = df69[ordem]

df70 = pd.merge(df68, df69, how='left', on=['ano', 'sigla_uf'])

#décima primeira tabela: Tráfico de entorpecentes e posse e uso de entorpocentes (T41)
df71 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2019-2020.xlsx', sheet_name='T41', skiprows=4, dtype=str)
df71 = df71[df71['UF'].notna()]
ordem = ['ano', 'sigla_uf','quantidade_trafico_entorpecente','quantidade_posse_uso_entorpecente']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Tráfico de Entorpecentes':'quantidade_trafico_entorpecente',
          'Posse e Uso de Entorpecentes':'quantidade_posse_uso_entorpecente'}
df71.rename(columns=rename, inplace=True)

df71['quantidade_trafico_entorpecente'] = df71['quantidade_trafico_entorpecente'].replace('...', np.nan) 
df71['quantidade_posse_uso_entorpecente'] = df71['quantidade_posse_uso_entorpecente'].replace('...', np.nan) 
df71['quantidade_posse_uso_entorpecente'] = df71['quantidade_posse_uso_entorpecente'].replace('-', 0) 

df71['quantidade_trafico_entorpecente'] = df71['quantidade_trafico_entorpecente'].astype('float').astype('Int64') 
df71['quantidade_posse_uso_entorpecente'] = df71['quantidade_posse_uso_entorpecente'].astype('float').astype('Int64')
df71['ano'] = '2019'
df71 = df71[ordem]

df72 = pd.merge(df70, df71, how='left', on=['ano', 'sigla_uf'])
dfx3 = df72[df72['sigla_uf'] == 'DF'] 
dfx3['id_municipio'] = 5300108
df72[df72['sigla_uf'] == 'DF'] = dfx3

df72

,ano,sigla_uf,id_municipio,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,quantidade_mortes_policiais_confronto,...,quantidade_lesao_corporal_dolosa_violencia_domestica,quantidade_estupro,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_posse_ilegal_arma_de_fogo,quantidade_porte_ilegal_arma_de_fogo,quantidade_posse_ilegal_porte_ilegal_arma_de_fogo,quantidade_trafico_entorpecente,quantidade_posse_uso_entorpecente
0,2019,AL,2704302,288,3,2,293,38,12,2,...,614,323,602,665,1267,296,404,700,944,89
1,2019,CE,2304400,636,8,13,708,51,7,2,...,2443,604,2741,2582,5323,<NA>,<NA>,<NA>,3301,1104
2,2019,DF,5300108,422,25,6,461,8,1,0,...,3160,762,3425,5137,8562,244,578,997,2597,5576
3,2019,ES,3205309,73,0,4,80,3,3,0,...,126,125,246,595,841,5,80,85,524,187
4,2019,GO,5208707,265,4,16,420,135,32,0,...,1633,501,1024,2393,3417,<NA>,271,453,1488,3460
5,2019,MA,2111300,204,24,3,264,33,12,0,...,2091,275,1158,748,1906,<NA>,249,249,933,680
6,2019,MT,5103403,94,5,0,99,17,17,0,...,<NA>,292,986,1040,2026,81,137,218,754,410
7,2019,PA,1501402,329,16,4,456,107,23,10,...,1704,354,800,1007,1807,45,62,107,997,325
8,2019,PB,2507507,184,6,2,192,9,4,1,...,909,27,970,485,1455,92,263,355,317,142
9,2019,PE,2611606,457,22,4,483,10,2,1,...,1843,431,3061,1485,4546,105,206,311,1897,729


In [ ]:
df72.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2019.csv', index=False)

#2020

In [ ]:
#coleta dados de 2020
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T43)
df73 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T43', skiprows=4, dtype=str)
df73 = df73[df73['UF'].notna()]
ordem = ['ano', 'grupo', 'sigla_uf', 'id_municipio', 'quantidade_homicidio_doloso']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Grupos segundo qualidade dos dados (1)':'grupo', 'Homicídios dolosos':'quantidade_homicidio_doloso'}
df73.rename(columns=rename, inplace=True)
df73['capital'] = df73['capital'].str.replace(r"\(.*\)","") 
df73['capital'] = df73['capital'].str.strip() 
df73['grupo'] = df73['grupo'].ffill()
df73 = pd.merge(df73, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) 
df73['ano'] = '2020'
df73 = df73[ordem] 

#segunda tabela: latrocinio por número de vítimas (T44)
df74 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T44', skiprows=4, dtype=str)
df74 = df74[df74['UF'].notna()]
ordem = ['ano', 'sigla_uf', '' 'quantidade_latrocinio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Latrocínio':'quantidade_latrocinio'}
df74.rename(columns=rename, inplace=True)
df74.reset_index(inplace=True, drop=True) #Para resetar o index
df74['quantidade_latrocinio'] = df74['quantidade_latrocinio'].replace('-', 0) 
df74['quantidade_latrocinio'] = df74['quantidade_latrocinio'].astype(int)
df74['ano'] = '2020'
df74 = df74[ordem]

df75 = pd.merge(df73, df74, how='left', on=['ano', 'sigla_uf'])

#terceira tabela: lesão corporal seguida de morte (T45)
df76 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T45', skiprows=4, dtype=str)
df76 = df76[df76['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal seguida de morte':'quantidade_lesao_corporal_morte'}
df76.rename(columns=rename, inplace=True)
df76['quantidade_lesao_corporal_morte'] = df76['quantidade_lesao_corporal_morte'].replace('-', 0) 
df76['quantidade_lesao_corporal_morte'] = df76['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df76.reset_index(inplace=True, drop=True) 
df76['quantidade_lesao_corporal_morte'] = np.floor(pd.to_numeric(df76['quantidade_lesao_corporal_morte'], errors='coerce')).astype('Int64')

df76['ano'] = '2020'
df76 = df76[ordem]

df76 = pd.merge(df75, df76, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Mortes decorrentes de intervenções policiais e Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T46)
df77 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T46', skiprows=4, dtype=str)
df77 = df77[df77['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 
         'quantidade_mortes_intervencao_policial', 
         'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']

rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Mortes Violentas Intencionais - MVI ':'quantidade_mortes_violentas_intencionais',
          'Proporção de MDIP em relação às MVI':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
          'Morte Decorrente de Intervenções Policiais (MDIP) em serviço e fora de serviço':'quantidade_mortes_intervencao_policial',
          }
df77.rename(columns=rename, inplace=True)

df77['quantidade_mortes_violentas_intencionais'] = df77['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df77['quantidade_mortes_intervencao_policial'] = df77['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df77['quantidade_mortes_violentas_intencionais'] = df77['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df77['quantidade_mortes_intervencao_policial'] = df77['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df77['quantidade_mortes_violentas_intencionais'] = df77['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df77['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df77['quantidade_mortes_intervencao_policial'] = df77['quantidade_mortes_intervencao_policial'].astype('float').astype('Int64')

df77['ano'] = '2020'
df77 = df77[ordem]

df78 = pd.merge(df76, df77, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Policiais Civis e Militares Mortos em Situação de Confronto (T42)
df79 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T42', skiprows=4, dtype=str)
df79 = df79[df79['UF'].notna()]

ordem = ['ano', 'sigla_uf', 'quantidade_mortes_policiais_confronto']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Policiais Civis e Militares Mortos em Situação de Confronto':'quantidade_mortes_policiais_confronto'}
df79.rename(columns=rename, inplace=True)
df79['quantidade_mortes_policiais_confronto'] = df79['quantidade_mortes_policiais_confronto'].replace('-', 0) 
df79['quantidade_mortes_policiais_confronto'] = df79['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 
df79['quantidade_mortes_policiais_confronto'] = df79['quantidade_mortes_policiais_confronto'].astype('float').astype('Int64')

df79['ano'] = '2020'
df79 = df79[ordem]

df80 = pd.merge(df78, df79, how='left', on=['ano', 'sigla_uf'])

#sexta tabela: Feminicídios (T47) 
df81 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T47', skiprows=4, dtype=str)
df81 = df81[df81['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_feminicidio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Feminicídios':'quantidade_feminicidio'}
df81.rename(columns=rename, inplace=True)

df81['quantidade_feminicidio'] = df81['quantidade_feminicidio'].replace('-', 0) 
df81['quantidade_feminicidio'] = df81['quantidade_feminicidio'].replace('...', np.nan) 
df81['quantidade_feminicidio'] = df81['quantidade_feminicidio'].astype('float').astype('Int64') 

df81['ano'] = '2020'
df81 = df81[ordem]

df82 = pd.merge(df80, df81, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Lesão Corporal Dolosa - Violência doméstica (T48)
df83 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T48', skiprows=4, dtype=str)
df83 = df83[df83['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_dolosa_violencia_domestica']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Lesão corporal dolosa - violência doméstica':'quantidade_lesao_corporal_dolosa_violencia_domestica'}
df83.rename(columns=rename, inplace=True)

df83['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df83['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('-', 0) 
df83['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df83['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('...', np.nan) 
df83['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df83['quantidade_lesao_corporal_dolosa_violencia_domestica'].astype('float').astype('Int64') 

df83['ano'] = '2020'
df83 = df83[ordem]

df84 = pd.merge(df82, df83, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T49)
df85 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T49', skiprows=4, dtype=str)
df85 = df85[df85['UF'].notna()]
ordem = ['ano', 
        'sigla_uf',
        'quantidade_estupro']
rename = {'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Estupros (1)':'quantidade_estupro'
          }
df85.rename(columns=rename, inplace=True)
df85['quantidade_estupro'] = df85['quantidade_estupro'].replace('-', 0) 
df85['quantidade_estupro'] = df85['quantidade_estupro'].replace('...', np.nan) 
df85['quantidade_estupro'] = df85['quantidade_estupro'].astype('float').astype('Int64') 

df85['ano'] = '2020'
df85 = df85[ordem]
df86 = pd.merge(df84, df85, how='left', on=['ano', 'sigla_uf'])

#nona tabela: Roubos e Furtos de veículos (T50)
df87 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T50', skiprows=4, dtype=str)
df87 = df87[df87['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_roubo_veiculos', 
          'quantidade_furto_veiculos',
          'quantidade_roubo_furto_veiculos']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Roubo de veículo':'quantidade_roubo_veiculos',
          'Furto de veículo':'quantidade_furto_veiculos',
          'Roubo e furto de veículo':'quantidade_roubo_furto_veiculos'}
df87.rename(columns=rename, inplace=True)

df87['quantidade_roubo_veiculos'] = df87['quantidade_roubo_veiculos'].replace('...', np.nan) 
df87['quantidade_furto_veiculos'] = df87['quantidade_furto_veiculos'].replace('...', np.nan) 
df87['quantidade_roubo_furto_veiculos'] = df87['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df87['quantidade_roubo_veiculos'] = df87['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df87['quantidade_furto_veiculos'] = df87['quantidade_furto_veiculos'].astype('float').astype('Int64')
df87['quantidade_roubo_furto_veiculos'] = df87['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')
df87['ano'] = '2020'
df87 = df87[ordem]

df88 = pd.merge(df86, df87, how='left', on=['ano', 'sigla_uf'])

#décima tabela: Posse e porte ilegal de arma de fogo (T51)
df89 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T51', skiprows=4, dtype=str)
df89 = df89[df89['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_posse_ilegal_arma_de_fogo', 
          'quantidade_porte_ilegal_arma_de_fogo',
          'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Posse ilegal de arma de fogo':'quantidade_posse_ilegal_arma_de_fogo',
          'Porte ilegal de arma de fogo':'quantidade_porte_ilegal_arma_de_fogo',
          'Posse e porte ilegal de arma de fogo':'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'}
df89.rename(columns=rename, inplace=True)

df89['quantidade_posse_ilegal_arma_de_fogo'] = df89['quantidade_posse_ilegal_arma_de_fogo'].replace('...', np.nan) 
df89['quantidade_porte_ilegal_arma_de_fogo'] = df89['quantidade_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 
df89['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df89['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 

df89['quantidade_posse_ilegal_arma_de_fogo'] = df89['quantidade_posse_ilegal_arma_de_fogo'].astype('float').astype('Int64') 
df89['quantidade_porte_ilegal_arma_de_fogo'] = df89['quantidade_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df89['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df89['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df89['ano'] = '2020'
df89 = df89[ordem]

df90 = pd.merge(df88, df89, how='left', on=['ano', 'sigla_uf'])

#décima primeira tabela: Tráfico de entorpecentes e posse e uso de entorpocentes (T52)
df91 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T52', skiprows=4, dtype=str)
df91 = df91[df91['UF'].notna()]
ordem = ['ano', 'sigla_uf','quantidade_trafico_entorpecente','quantidade_posse_uso_entorpecente']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Tráfico de Entorpecentes':'quantidade_trafico_entorpecente',
          'Posse e Uso de Entorpecentes':'quantidade_posse_uso_entorpecente'}
df91.rename(columns=rename, inplace=True)

df91['quantidade_trafico_entorpecente'] = df91['quantidade_trafico_entorpecente'].replace('...', np.nan) 
df91['quantidade_posse_uso_entorpecente'] = df91['quantidade_posse_uso_entorpecente'].replace('...', np.nan) 
df91['quantidade_posse_uso_entorpecente'] = df91['quantidade_posse_uso_entorpecente'].replace('-', 0) 

df91['quantidade_trafico_entorpecente'] = df91['quantidade_trafico_entorpecente'].astype('float').astype('Int64') 
df91['quantidade_posse_uso_entorpecente'] = df91['quantidade_posse_uso_entorpecente'].astype('float').astype('Int64')
df91['ano'] = '2020'
df91 = df91[ordem]

df92 = pd.merge(df90, df91, how='left', on=['ano', 'sigla_uf'])

dfx4 = df92[df92['sigla_uf'] == 'DF'] 
dfx4['id_municipio'] = 5300108
df92[df92['sigla_uf'] == 'DF'] = dfx4

df92.head()


,ano,grupo,sigla_uf,id_municipio,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,...,quantidade_lesao_corporal_dolosa_violencia_domestica,quantidade_estupro,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_posse_ilegal_arma_de_fogo,quantidade_porte_ilegal_arma_de_fogo,quantidade_posse_ilegal_porte_ilegal_arma_de_fogo,quantidade_trafico_entorpecente,quantidade_posse_uso_entorpecente
0,2020,Grupo 1,AL,2704302,362,5,3,370,4,1,...,519,287,493,755,1248,194,329,523,1001,40
1,2020,Grupo 1,CE,2304400,1216,12,18,1303,57,4,...,129,554,4969,2591,7560,172,442,614,2625,799
2,2020,Grupo 1,DF,5300108,376,33,5,425,11,2,...,3276,699,2218,4285,6503,264,573,1006,2993,5403
3,2020,Grupo 1,ES,3205309,67,0,4,79,8,10,...,155,107,154,669,823,5,67,72,498,107
4,2020,Grupo 1,GO,5208707,233,5,9,382,135,35,...,1622,458,608,1606,2214,<NA>,300,416,1556,6982


In [ ]:
df92.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2020.csv', index=False)

#2021

In [ ]:
#coleta dados de 2021
#primeira tabela: homicidios dolosos por numero de vítimas (capitais) (T43)
df93 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T43', skiprows=4, dtype=str)
df93 = df93[df93['UF'].notna()]
ordem = ['ano', 'grupo', 'sigla_uf', 'id_municipio', 'quantidade_homicidio_doloso']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Grupos segundo qualidade dos dados (1)':'grupo', 'Unnamed: 4':'quantidade_homicidio_doloso'}
df93.rename(columns=rename, inplace=True)
df93['capital'] = df93['capital'].str.replace(r"\(.*\)","") 
df93['capital'] = df93['capital'].str.strip() 
df93['grupo'] = df93['grupo'].ffill()
df93 = pd.merge(df93, municipio, how='left', left_on=['sigla_uf', 'capital'], right_on=['sigla_uf', 'nome']) 
df93['ano'] = '2021'
df93 = df93[ordem] 

#segunda tabela: latrocinio por número de vítimas (T44)
df94 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T44', skiprows=4, dtype=str)
df94 = df94[df94['UF'].notna()]
ordem = ['ano', 'sigla_uf', '' 'quantidade_latrocinio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Unnamed: 4':'quantidade_latrocinio'}
df94.rename(columns=rename, inplace=True)
df94.reset_index(inplace=True, drop=True) 
df94['quantidade_latrocinio'] = df94['quantidade_latrocinio'].replace('-', 0) 
df94['quantidade_latrocinio'] = df94['quantidade_latrocinio'].astype(int)
df94['ano'] = '2021'
df94 = df94[ordem]

df95 = pd.merge(df93, df94, how='left', on=['ano', 'sigla_uf'])

#terceira tabela: lesão corporal seguida de morte (T45)
df96 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T45', skiprows=4, dtype=str)
df96 = df96[df96['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_morte']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Unnamed: 4':'quantidade_lesao_corporal_morte'}
df96.rename(columns=rename, inplace=True)
df96['quantidade_lesao_corporal_morte'] = df96['quantidade_lesao_corporal_morte'].replace('-', 0) 
df96['quantidade_lesao_corporal_morte'] = df96['quantidade_lesao_corporal_morte'].replace('...', np.nan) 
df96.reset_index(inplace=True, drop=True) 
df96['quantidade_lesao_corporal_morte'] = np.floor(pd.to_numeric(df96['quantidade_lesao_corporal_morte'], errors='coerce')).astype('Int64')

df96['ano'] = '2021'
df96 = df96[ordem]

df96 = pd.merge(df95, df96, how='left', on=['ano', 'sigla_uf'])

#quarta tabela: Mortes decorrentes de intervenções policiais e Proporção de Mortes decorrentes de intervenções policiais em relação às Mortes Violentas Intencionais (T46)
df97 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T46', skiprows=4, dtype=str)
df97 = df97[df97['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_mortes_violentas_intencionais', 
         'quantidade_mortes_intervencao_policial', 
         'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais']

rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Unnamed: 4':'quantidade_mortes_violentas_intencionais',
          'Unnamed: 8':'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
          'Unnamed: 6':'quantidade_mortes_intervencao_policial',
          }
df97.rename(columns=rename, inplace=True)

df97['quantidade_mortes_violentas_intencionais'] = df97['quantidade_mortes_violentas_intencionais'].replace('-', 0) 
df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('-', 0)
df97['quantidade_mortes_intervencao_policial'] = df97['quantidade_mortes_intervencao_policial'].replace('-', 0) 

df97['quantidade_mortes_violentas_intencionais'] = df97['quantidade_mortes_violentas_intencionais'].replace('...', np.nan) 
df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'].replace('...', np.nan)
df97['quantidade_mortes_intervencao_policial'] = df97['quantidade_mortes_intervencao_policial'].replace('...', np.nan) 

df97['quantidade_mortes_violentas_intencionais'] = df97['quantidade_mortes_violentas_intencionais'].astype('float').astype('Int64')
df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'] = np.floor(pd.to_numeric(df97['proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais'], errors='coerce')).astype('Int64')
df97['quantidade_mortes_intervencao_policial'] = df97['quantidade_mortes_intervencao_policial'].astype('float').astype('Int64')

df97['ano'] = '2021'
df97 = df97[ordem]

df98 = pd.merge(df96, df97, how='left', on=['ano', 'sigla_uf'])

#quinta tabela: Policiais Civis e Militares Mortos em Situação de Confronto (T42)
df99 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T42', skiprows=4, dtype=str)
df99 = df99[df99['UF'].notna()]

ordem = ['ano', 'sigla_uf', 'quantidade_mortes_policiais_confronto']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Unnamed: 10':'quantidade_mortes_policiais_confronto'}
df99.rename(columns=rename, inplace=True)
df99['quantidade_mortes_policiais_confronto'] = df99['quantidade_mortes_policiais_confronto'].replace('-', 0) 
df99['quantidade_mortes_policiais_confronto'] = df99['quantidade_mortes_policiais_confronto'].replace('...', np.nan) 
df99['quantidade_mortes_policiais_confronto'] = df99['quantidade_mortes_policiais_confronto'].astype('float').astype('Int64')

df99['ano'] = '2021'
df99 = df99[ordem]

df100 = pd.merge(df98, df99, how='left', on=['ano', 'sigla_uf'])

#sexta tabela: Feminicídios (T47) 
df101 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T47', skiprows=4, dtype=str)
df101 = df101[df101['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_feminicidio']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Unnamed: 3':'quantidade_feminicidio'}
df101.rename(columns=rename, inplace=True)

df101['quantidade_feminicidio'] = df101['quantidade_feminicidio'].astype('float').astype('Int64') 

df101['ano'] = '2021'
df101 = df101[ordem]

df102 = pd.merge(df100, df101, how='left', on=['ano', 'sigla_uf'])

#sétima tabela: Lesão Corporal Dolosa - Violência doméstica (T48)
df103 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T48', skiprows=4, dtype=str)
df103 = df103[df103['UF'].notna()]
ordem = ['ano', 'sigla_uf', 'quantidade_lesao_corporal_dolosa_violencia_domestica']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 'Unnamed: 3':'quantidade_lesao_corporal_dolosa_violencia_domestica'}
df103.rename(columns=rename, inplace=True)

df103['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df103['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('-', 0) 
df103['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df103['quantidade_lesao_corporal_dolosa_violencia_domestica'].replace('...', np.nan) 
df103['quantidade_lesao_corporal_dolosa_violencia_domestica'] = df103['quantidade_lesao_corporal_dolosa_violencia_domestica'].astype('float').astype('Int64') 

df103['ano'] = '2021'
df103 = df103[ordem]

df104 = pd.merge(df102, df103, how='left', on=['ano', 'sigla_uf'])

#oitava tabela: Estupros (T49)
df105 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T49', skiprows=4, dtype=str)
df105 = df105[df105['UF'].notna()]
ordem = ['ano', 
        'sigla_uf',
        'quantidade_estupro']
rename = {'UF':'sigla_uf', 
          'Capitais e Distrito Federal':'capital', 
          'Unnamed: 3':'quantidade_estupro'
          }
df105.rename(columns=rename, inplace=True)
df105['quantidade_estupro'] = df105['quantidade_estupro'].replace('-', 0) 
df105['quantidade_estupro'] = df105['quantidade_estupro'].replace('...', np.nan) 
df105['quantidade_estupro'] = df105['quantidade_estupro'].astype('float').astype('Int64') 

df105['ano'] = '2021'
df105 = df105[ordem]
df106 = pd.merge(df104, df105, how='left', on=['ano', 'sigla_uf'])

#nona tabela: Roubos e Furtos de veículos (T50)
df107 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T50', skiprows=4, dtype=str)
df107 = df107[df107['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_roubo_veiculos', 
          'quantidade_furto_veiculos',
          'quantidade_roubo_furto_veiculos']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Unnamed: 3':'quantidade_roubo_veiculos',
          'Unnamed: 8':'quantidade_furto_veiculos',
          'Unnamed: 13':'quantidade_roubo_furto_veiculos'}
df107.rename(columns=rename, inplace=True)

df107['quantidade_roubo_veiculos'] = df107['quantidade_roubo_veiculos'].replace('...', np.nan) 
df107['quantidade_furto_veiculos'] = df107['quantidade_furto_veiculos'].replace('...', np.nan) 
df107['quantidade_roubo_furto_veiculos'] = df107['quantidade_roubo_furto_veiculos'].replace('...', np.nan) 

df107['quantidade_roubo_veiculos'] = df107['quantidade_roubo_veiculos'].astype('float').astype('Int64') 
df107['quantidade_furto_veiculos'] = df107['quantidade_furto_veiculos'].astype('float').astype('Int64')
df107['quantidade_roubo_furto_veiculos'] = df107['quantidade_roubo_furto_veiculos'].astype('float').astype('Int64')
df107['ano'] = '2021'
df107 = df107[ordem]

df108 = pd.merge(df106, df107, how='left', on=['ano', 'sigla_uf'])

#décima tabela: Posse e porte ilegal de arma de fogo (T51)
df109 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T51', skiprows=4, dtype=str)
df109 = df109[df109['UF'].notna()]
ordem = ['ano', 'sigla_uf',
          'quantidade_posse_ilegal_arma_de_fogo', 
          'quantidade_porte_ilegal_arma_de_fogo',
          'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Unnamed: 3':'quantidade_posse_ilegal_arma_de_fogo',
          'Unnamed: 5':'quantidade_porte_ilegal_arma_de_fogo',
          'Unnamed: 7':'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'}
df109.rename(columns=rename, inplace=True)

df109['quantidade_posse_ilegal_arma_de_fogo'] = df109['quantidade_posse_ilegal_arma_de_fogo'].replace('...', np.nan) 
df109['quantidade_porte_ilegal_arma_de_fogo'] = df109['quantidade_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 
df109['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df109['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].replace('...', np.nan) 

df109['quantidade_posse_ilegal_arma_de_fogo'] = df109['quantidade_posse_ilegal_arma_de_fogo'].astype('float').astype('Int64') 
df109['quantidade_porte_ilegal_arma_de_fogo'] = df109['quantidade_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df109['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'] = df109['quantidade_posse_ilegal_porte_ilegal_arma_de_fogo'].astype('float').astype('Int64')
df109['ano'] = '2021'
df109 = df109[ordem]

df110 = pd.merge(df108, df109, how='left', on=['ano', 'sigla_uf'])

#décima primeira tabela: Tráfico de entorpecentes e posse e uso de entorpocentes (T52)
df111 = pd.read_excel('/content/gdrive/MyDrive/datascience/basedosdados/input/anuario_2020-2021.xlsx', sheet_name='T52', skiprows=4, dtype=str)
df111 = df111[df111['UF'].notna()]
ordem = ['ano', 'sigla_uf','quantidade_trafico_entorpecente','quantidade_posse_uso_entorpecente']
rename = {'UF':'sigla_uf', 'Capitais e Distrito Federal':'capital', 
          'Unnamed: 3':'quantidade_trafico_entorpecente',
          'Unnamed: 8':'quantidade_posse_uso_entorpecente'}
df111.rename(columns=rename, inplace=True)

df111['quantidade_trafico_entorpecente'] = df111['quantidade_trafico_entorpecente'].replace('...', np.nan) 
df111['quantidade_posse_uso_entorpecente'] = df111['quantidade_posse_uso_entorpecente'].replace('...', np.nan) 
df111['quantidade_posse_uso_entorpecente'] = df111['quantidade_posse_uso_entorpecente'].replace('-', 0) 

df111['quantidade_trafico_entorpecente'] = df111['quantidade_trafico_entorpecente'].astype('float').astype('Int64') 
df111['quantidade_posse_uso_entorpecente'] = df111['quantidade_posse_uso_entorpecente'].astype('float').astype('Int64')
df111['ano'] = '2021'
df111 = df111[ordem]

df112 = pd.merge(df110, df111, how='left', on=['ano', 'sigla_uf'])

dfx5 = df112[df112['sigla_uf'] == 'DF'] 
dfx5['id_municipio'] = 5300108
df112[df112['sigla_uf'] == 'DF'] = dfx5
df112.head()


,ano,grupo,sigla_uf,id_municipio,quantidade_homicidio_doloso,quantidade_latrocinio,quantidade_lesao_corporal_morte,quantidade_mortes_violentas_intencionais,quantidade_mortes_intervencao_policial,proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais,...,quantidade_lesao_corporal_dolosa_violencia_domestica,quantidade_estupro,quantidade_roubo_veiculos,quantidade_furto_veiculos,quantidade_roubo_furto_veiculos,quantidade_posse_ilegal_arma_de_fogo,quantidade_porte_ilegal_arma_de_fogo,quantidade_posse_ilegal_porte_ilegal_arma_de_fogo,quantidade_trafico_entorpecente,quantidade_posse_uso_entorpecente
0,2021,Grupo 1,AL,2704302,304,2,1,307,4,1,...,628,287,390,804,1194,175,301,476,1157,77
1,2021,Grupo 1,CE,2304400,871,19,5,927,32,3,...,78,547,4158,2313,6471,131,393,524,2498,749
2,2021,Grupo 1,DF,5300108,310,23,6,347,8,2,...,3127,593,2030,3733,5763,256,510,915,2694,5192
3,2021,Grupo 1,ES,3205309,66,1,2,78,9,11,...,139,72,241,537,778,7,80,87,463,180
4,2021,Grupo 1,GO,5208707,165,7,3,258,83,32,...,1260,409,335,1436,1771,<NA>,254,334,1301,11964


In [ ]:
df112.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2021.csv', index=False)

#Juntando as tabelas em um arquivo 


In [ ]:
df_append = pd.DataFrame()

years = [2016, 2017, 2018, 2019, 2020, 2021]

csv_urls = []

column_order = [
  'id_municipio',
  'grupo',
  'sigla_uf',
  'ano',
  'quantidade_homicidio_doloso',
  'quantidade_latrocinio',
  'quantidade_lesao_corporal_morte',
  'quantidade_mortes_policiais_confronto',
  'quantidade_mortes_intervencao_policial',
  'quantidade_mortes_intervencao_policial_civil_em_servico',
  'quantidade_mortes_intervencao_policial_militar_em_servico',
  'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
  'quantidade_mortes_intervencao_policial_militar_fora_de_servico',
  'quantidade_mortes_violentas_intencionais',
  'quantidade_feminicidio',
  'quantidade_lesao_corporal_dolosa_violencia_domestica',
  'quantidade_estupro',
  'quantidade_roubo_veiculos',
  'quantidade_furto_veiculos',
  'quantidade_roubo_furto_veiculos',
  'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
  'quantidade_posse_ilegal_arma_de_fogo',
  'quantidade_porte_ilegal_arma_de_fogo',
  'quantidade_posse_ilegal_porte_ilegal_arma_de_fogo',
  'quantidade_trafico_entorpecente',
  'quantidade_posse_uso_entorpecente',
  'quantidade_morte_policiais_civis_confronto_em_servico',
  'quantidade_morte_policiais_militares_confronto_em_servico',
  'quantidade_morte_policiais_civis_fora_de_servico',
  'quantidade_morte_policiais_militares_fora_de_servico'
]

for i in years:
  csv_url = f'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_{i}.csv'
  csv_urls.append(csv_url)

df_concat = pd.concat([pd.read_csv(file) for file in csv_urls])

df_concat = df_concat[column_order]

df_concat.to_csv(r'/content/gdrive/MyDrive/datascience/basedosdados/output/anuario_2016_2021.csv', index=False)

In [ ]:
df_concat.columns

Index(['id_municipio', 'grupo', 'sigla_uf', 'ano',
       'quantidade_homicidio_doloso', 'quantidade_latrocinio',
       'quantidade_lesao_corporal_morte',
       'quantidade_mortes_policiais_confronto',
       'quantidade_mortes_intervencao_policial',
       'quantidade_mortes_intervencao_policial_civil_em_servico',
       'quantidade_mortes_intervencao_policial_militar_em_servico',
       'quantidade_mortes_intervencao_policial_civil_fora_de_servico',
       'quantidade_mortes_intervencao_policial_militar_fora_de_servico',
       'quantidade_mortes_violentas_intencionais', 'quantidade_feminicidio',
       'quantidade_lesao_corporal_dolosa_violencia_domestica',
       'quantidade_estupro', 'quantidade_roubo_veiculos',
       'quantidade_furto_veiculos', 'quantidade_roubo_furto_veiculos',
       'proporcao_mortes_intenvencao_policial_x_mortes_violentas_intencionais',
       'quantidade_posse_ilegal_arma_de_fogo',
       'quantidade_porte_ilegal_arma_de_fogo',
       'quantidade_posse_i